# Stress 2D

Using Prandtl formulation to slove stress with torsion by hand-mesh FEM.

## Module

In [1]:
import numpy as np


class Boundary:
    def __init__(self, node):
        self.node = node

    def getMat(self):
        pass

    def getLoad(self):
        pass

    def getBoundOn(self, ele):
        if self.node[0] in ele.node and self.node[1] in ele.node:
            a, b = ele.node.index(self.node[0]), ele.node.index(self.node[1])
            if b < a:
                return [b, a]
            return [a, b]
        return None


class Stress(Boundary):
    def __init__(self, n_ele, g, th):
        self.ele = elements[n_ele]
        super().__init__(self.ele.node)
        self.stress = g * th

    def getMat(self):
        return 0

    def getLoad(self):
        l = len(self.node)
        return self.stress * self.ele.getArea() * np.ones(l) * 2 / l

    def getBoundOn(self, ele):
        if all([(n in ele.node) for n in self.node]):
            return np.arange(len(self.node))
        else:
            return None


class Element:
    def __init__(self, node):
        self.node = node
        self.load = np.zeros(len(node))
        self.matrix = np.zeros([len(node), len(node)])

    def writeGlobal(self, mat, load):
        mat[np.ix_(self.node, self.node)] += self.matrix
        load[self.node] += self.load

    def setBound(self, bound):
        ind = bound.getBoundOn(self)
        self.matrix[np.ix_(ind, ind)] += bound.getMat()
        self.load[ind] = bound.getLoad()

    def getArea(self):
        return 1


class Square(Element):
    def __init__(self, n1, n2, n3, n4, kx, ky):
        """
        Define the square
        n4 n3
        n1 n2
        """
        super().__init__([n1, n2, n3, n4])

        # prepare matrix
        a = np.array([2, -2, -1, 1])
        mx = np.stack([a, -a, -np.flip(a), np.flip(a)])
        my = mx.copy()
        my[mx ==  1] = -2
        my[mx == -2] =  1

        # get length of rectangle
        l12 = getLen(n1, n2)
        l23 = getLen(n2, n3)

        # calculate condutance
        self.matrix = (kx * l23 / l12 * mx + ky * l12 / l23 * my) / 6

    def getArea(self):
        pos = nodes[self.node]
        dpos = np.roll(pos, -1, axis=0) - np.roll(pos, -2, axis=0)
        return np.abs(np.cross(dpos[0], dpos[1]))


class Triangle(Element):
    def __init__(self, n1, n2, n3, kx, ky):
        """
        Define the triangle
            n3
        n1     n2
        """
        super().__init__([n1, n2, n3])

        # prepare matrix
        pos = nodes[self.node]
        dpos = np.roll(pos, -1, axis=0) - np.roll(pos, -2, axis=0)
        dpos[:, 0] *= -1
        self.matrix = kx * np.outer(dpos[:, 1], dpos[:, 1]) \
                    + ky * np.outer(dpos[:, 0], dpos[:, 0])
        self.matrix = self.matrix / 4 / self.getArea()

    def getArea(self):
        pos = nodes[self.node]
        dpos = np.roll(pos, -1, axis=0) - np.roll(pos, -2, axis=0)
        return np.cross(dpos[0], dpos[1]) / 2


def getLen(n1, n2):
    return np.sqrt(((nodes[n1] - nodes[n2]) ** 2).sum())


## Custom data
![](https://raw.githubusercontent.com/linnil1/2019FEM/master/images/Stress_2D.png)

* nodes: position of nodes, is a list of (x, y)
* elements: One of Square and Triangle
    * Square parameters $(node_i, node_j, node_m, node_n, k_x, k_y)$ ($k_x$, $k_y$ should = 1 in this case)
    * Triangle parameters $(node_i, node_j, node_k, k_x, k_y)$ ($k_x$, $k_y$ should = 1 in this case)
* boundarys: Stress
    * Stress parameters (elementId, G, $\theta$)
* Const:
    * Each tuple in the list has two values $(node_i, T_i)$,
      which means $T(node_i) = T_i$

In [2]:
nodes = [[0, 0],
    [0, 0], [0.125, 0   ], [0.25, 0],
            [0.125, 0.25], [0.25, .25],
                           [0.25, .5],
]
nodes = np.array(nodes)

elements = [
    Triangle(1, 2, 4, 1, 1),
    Square(2, 3, 5, 4, 1, 1),
    Triangle(4, 5, 6, 1, 1)
]

boundarys = [
    Stress(0, 11e6, 0.0005),
    Stress(1, 11e6, 0.0005),
    Stress(2, 11e6, 0.0005),
]

Const = [
    [3, 0],
    [5, 0],
    [6, 0],
]

np.set_printoptions(3)

## solve it

In [3]:
n = len(nodes)
T = np.zeros([n, n])
L = np.zeros([n])

for ele in elements:
    print("ele")
    print(ele.matrix)
    for bound in boundarys:
        if np.any(bound.getBoundOn(ele)):
            ele.setBound(bound)
            print("bound")
            print(ele.matrix)
    ele.writeGlobal(T, L)

print("Table Load")
print(T[1:, 1:])
print(L[1:])

for c in Const:
    T[c[0], :] = 0
    T[c[0], c[0]] = 1
    L[c[0]] = c[1]

print("Table Load with Boundary")
print(T[1:, 1:])
print(L[1:])

result = np.linalg.inv(T[1:, 1:]).dot(L[1:])
print("Result")
print(result)

ele
[[ 1.   -1.   -0.  ]
 [-1.    1.25 -0.25]
 [-0.   -0.25  0.25]]
bound
[[ 1.   -1.    0.  ]
 [-1.    1.25 -0.25]
 [ 0.   -0.25  0.25]]
ele
[[ 0.833 -0.583 -0.417  0.167]
 [-0.583  0.833  0.167 -0.417]
 [-0.417  0.167  0.833 -0.583]
 [ 0.167 -0.417 -0.583  0.833]]
bound
[[ 0.833 -0.583 -0.417  0.167]
 [-0.583  0.833  0.167 -0.417]
 [-0.417  0.167  0.833 -0.583]
 [ 0.167 -0.417 -0.583  0.833]]
ele
[[ 1.   -1.   -0.  ]
 [-1.    1.25 -0.25]
 [-0.   -0.25  0.25]]
bound
[[ 1.   -1.    0.  ]
 [-1.    1.25 -0.25]
 [ 0.   -0.25  0.25]]
Table Load
[[ 1.    -1.     0.     0.     0.     0.   ]
 [-1.     2.083 -0.583 -0.083 -0.417  0.   ]
 [ 0.    -0.583  0.833 -0.417  0.167  0.   ]
 [ 0.    -0.083 -0.417  2.083 -1.583  0.   ]
 [ 0.    -0.417  0.167 -1.583  2.083 -0.25 ]
 [ 0.     0.     0.     0.    -0.25   0.25 ]]
[ 57.292 143.229  85.938 200.521 143.229  57.292]
Table Load with Boundary
[[ 1.    -1.     0.     0.     0.     0.   ]
 [-1.     2.083 -0.583 -0.083 -0.417  0.   ]
 [ 0.     0.     

## postprocessing
Calculate the stress (Only triangle)

In [4]:
for i, ele in enumerate(elements):
    if type(ele) is not Triangle:
        continue
    node = np.array(ele.node)
    pos = nodes[node]
    dpos = np.roll(pos, -1, axis=0) - np.roll(pos, -2, axis=0)
    dpos[:, 0] *= -1
    A = ele.getArea()
    tou_zx = dpos[:, 0].dot(result[node - 1]) / 2 / A
    tou_zy = dpos[:, 1].dot(result[node - 1]) / 2 / A * -1
    print(dpos)
    print(i)
    print(dpos[:, 0])
    print("tou zx", tou_zx)
    print("tou zy", tou_zy)

[[-0.    -0.25 ]
 [-0.125  0.25 ]
 [ 0.125  0.   ]]
0
[-0.    -0.125  0.125]
tou zx -356.48148148148147
tou zy 458.33333333333326
[[-0.    -0.25 ]
 [-0.125  0.25 ]
 [ 0.125  0.   ]]
2
[-0.    -0.125  0.125]
tou zx 0.0
tou zy 831.79012345679
